# Questão 1

Classes:
    - Promotor
    - Detrator
    - Neutro
Só 100 notícias são classificadas
Os conteúdos são rankeados por alcance
    - No grupo com os 50 maiores alcances, todos são classificados manualmente
    - O restante, 50 são selecionados aleatoriamentes e classificados manualmente
    - Alcance por grupo
    
Calcular margem de erro do índice
    

In [40]:
import cmath

In [12]:
alcance_promotor_top = 1000000 * (15/50)
alcance_detrator_top = 1000000 * (8/50)

In [13]:
alcance_promotor_res = 3000000 * (17/50)
alcance_detrator_res = 3000000 * (7/50)

In [20]:
alcance_promotor_total = alcance_promotor_top + alcance_promotor_res
alcance_detrator_total = alcance_detrator_top + alcance_detrator_res

In [23]:
indice_promotor_top = alcance_promotor_total / (alcance_promotor_top + alcance_detrator_top)
indice_promotor_res = alcance_promotor_total / (alcance_promotor_res+alcance_detrator_res)

In [24]:
indice_promotor_top

2.869565217391304

In [25]:
indice_promotor_res

0.9166666666666665

In [39]:
def margem_erro(populacao, amostra, p=0.5, z = 1.96):
    '''
    p: percentual (como não tem definido, escolhe 50% que é a pior)
    z: valor referente a tabela de distribuição da normal (referente a 95% de confiança)
    '''
    return cmath.sqrt(-(z**2*p*(1-p)*(amostra-populacao))/(amostra*populacao)).real

In [50]:
#margem de erro TOP 50
margem_erro(50,50)

0.0

In [51]:
#margem de erro Restante
margem_erro(550,50)

0.13214317304279544

In [52]:
#margem de erro de amostra 100 da população de 600
margem_erro(600,100)

0.08946135105917713

# Resposta Questão 1

A geração do índice depende do alcance de cada grupo. Para verificar a margem de erro do índice, a margem de erro dos grupos deve ser levado em consideração.

Utilizando a fórmula para descobrir o tamanho da amostra (https://pt.wikihow.com/Calcular-o-Tamanho-de-uma-Amostra) os dados de cada amostra foram utilizadas. (obs: 95% de confiança foi utilizado)

Para o grupo Top 50, como é um grupo controlado onde a população e amostra podem ser consideradas iguais, a margem de erro é 0. Já o restante, a marge de erro é cerca de 13% pois utilizo uma amostra de 50 de uma população de 550. Novamente, como o índice utiliza de ambos o grupo para compor o valor, essa margem será passada para o índice.

Se ao invés de segmentar utilizasse uma amostra de 100, a margem de erro seria menor (cerca de 8.95%) o que seria melhor para o índice.

Link de calculadoras:
- https://www.solvis.com.br/calculos-de-amostragem/
- https://pt.surveymonkey.com/mp/margin-of-error-calculator/
- https://www.opinionbox.com/calculadora-margem-de-erro/

# Questão 4

Agrupar notícias por assunto

In [251]:
import pandas as pd

pd.options.display.max_rows
pd.set_option('display.max_colwidth', 100)

In [85]:
#leitura do arquivo
df = pd.read_csv(filepath_or_buffer='agrupamento_eventos_new.csv',delimiter='\t',encoding='utf-8')

In [ ]:
df.head()

In [87]:
#verificando as datas únicas
df.Data.unique()

array(['2020-01-22', '2020-01-29', '2020-01-08', '2020-01-06',
       '2020-01-28', '2020-01-02', '2020-01-23', '2020-01-07',
       '2020-01-20', '2020-01-15', '2020-01-01', '2020-01-14',
       '2020-01-17', '2020-01-27', '2020-01-13', '2020-01-16',
       '2020-01-09', '2020-01-21', '2020-01-10', '2020-01-03',
       '2020-01-18', '2020-01-26', '2020-01-19', '2020-01-12',
       '2020-01-05', '2020-01-24', '2020-01-11', '2020-01-04',
       '2020-01-25'], dtype=object)

In [88]:
#ordenando pela data
df = df.sort_values(by=['Data'])

In [89]:
#transformando data em datetime
df['Data'] = pd.to_datetime(df['Data'])

In [157]:
#import de bibliotecas
from sklearn.feature_extraction.text import CountVectorizer
from nltk import corpus
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

#stopwords em portugues + duas palavras que não estavam incluídas
stopwords_pt = corpus.stopwords.words('portuguese')
stopwords_pt.append('ainda')
stopwords_pt.append('ser')

In [158]:
#geração do vocabulário, onde seleciono apenas palavras que aparecem em menos de 80% no documento e
#em pelo menos 2 documentos diferentes. Também são retirados as stopwords e consideram apenas palavras com mais 
#de 3 letras. Todas as palavras são colocadas em caixa baixa
def vocab_generation(df):
    count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words=stopwords_pt, lowercase=True, token_pattern=r'(?u)\b[a-zA-Z_.]{3,}\b')
    doc_term_matrix = count_vect.fit_transform(df)
    return doc_term_matrix, count_vect

In [55]:
#busca pelos melhores parâmetros para o algoritmo. Quantidade de topicos e taxa de aprendizado
def best_param(df,n_topic=[3,4,5,6,7,8,9],learning_decay=[.5, .7, .9]):
    search_params = {'n_components': n_topic, 'learning_decay': learning_decay}
    lda = LatentDirichletAllocation()
    model = GridSearchCV(lda, param_grid=search_params,n_jobs=2)
    model.fit(df)
    return model.best_params_

In [42]:
#Modelo LDA
def lda_model(df,n_components_, learning_decay_):
    LDA = LatentDirichletAllocation(n_components=n_components_,learning_decay =learning_decay_, random_state=7)
    LDA.fit(df)
    return LDA

In [242]:
#função que recebe o dataframe, com data de início e fim, e um apelido. Retorna o dataframe com a coluna de topico
def topic_classification(df, range_start, range_end, alias):
    df_ = df.loc[(df['Data'] >= range_start) & (df['Data'] <= range_end)]
    vocab_, count_vect_ = vocab_generation(df_['Conteúdo'].values.astype('U'))
    best_param_ = best_param(vocab_)
    lda_ = lda_model(vocab_,best_param_['n_components'],best_param_['learning_decay'])
    topic_values_ = lda_.transform(vocab_)
    
    dict_={}
    for i,topic in enumerate(lda_.components_):
        dict_[i]=[count_vect_.get_feature_names()[i] for i in topic.argsort()[-3:]]
    
    df_['Topic'] = topic_values_.argmax(axis=1)
    df_['Topic'] = df_['Topic'].astype(int)
    df_['Topic']= df_['Topic'].apply(lambda x: dict_.get(x,x))
    df_['Topic']= df_['Topic'].astype(str) + ' - ' + alias
    
    return df_

In [243]:
df_week1 = topic_classification(df, '2020-01-01', '2020-01-07', 'week-1')
df_week2 = topic_classification(df, '2020-01-08', '2020-01-14', 'week-2')
df_week3 = topic_classification(df, '2020-01-15', '2020-01-21', 'week-3')
df_week4 = topic_classification(df, '2020-01-22', '2020-01-31', 'week-4')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [244]:
df_final = pd.concat([df_week1, df_week2, df_week4, df_week4], ignore_index=True)

In [252]:
df_final.head()

,ID,Data,Título,Conteúdo,Veículo,Topic
0,51284,2020-01-01,Oferta de ações na bolsa brasileira pode atingir até R$ 200 bi este ano,As empresas brasileiras devem continuar se financiando na Bolsa para promover sua expansão em 20...,Correio Popular,"['alta', 'ibovespa', 'mercado'] - week-1"
1,51271,2020-01-01,JBS: acordo de acionistas entre J&F e BNDESPar perde validade,O acordo de acionistas da JBS com o Banco Nacional de Desenvolvimento Econômico e Social (BNDES)...,Correio Popular,"['bancos', 'consignado', 'banco'] - week-1"
2,51276,2020-01-01,JBS: acordo de acionistas entre J&F e BNDESPar perde validade,O acordo de acionistas da JBS com o Banco Nacional de Desenvolvimento Econômico e Social (BNDES)...,Revista Istoé,"['bancos', 'consignado', 'banco'] - week-1"
3,51277,2020-01-01,Oferta de ações na bolsa brasileira pode atingir até R$ 200 bi este ano,As empresas brasileiras devem continuar se financiando na Bolsa para promover sua expansão em 20...,UOL,"['alta', 'ibovespa', 'mercado'] - week-1"
4,51267,2020-01-01,Oferta de ações na Bolsa brasileira pode atingir até R$ 200 bi este ano,"Oferta de ações na Bolsa pode atingir R$ 200 bilhões \nA exemplo do que ocorreu em 2019, empresa...",Estadão,"['alta', 'ibovespa', 'mercado'] - week-1"


# Resposta Questão 4

Foi assumido que os assuntos duram apenas uma semana, para que os tópicos não ficassem enviesado. Assim, cada semana foi quebrada em um dataset e os tópicos foram analisados com uso do algoritmo não-supervisionado LDA.

A coluna 'Topic' no dataset final contém as 3 palavras mais relevante de cada tópico encontrado, bem como um alias para identificar a forma que eles apareceram.